In [1]:
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install mdatagen
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install numpy scipy sklearn
!pip install jax
!pip install chex
!pip install attr
!pip install optax



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from geomloss import SamplesLoss

import ot

import os
import pickle as pkl
import copy

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from utils import *
from SoftImpute import softimpute, cv_softimpute
#from DataSetLoader import dataset_loader, ground_truth
from SinkhornImputation import SinkhornImputation
from Sinkhorn_CMI import *
from RR_imputer import RRimputer
import matplotlib.pyplot as plt
#from CMI import *

from Inject_Missing_Values import *

from Miss import *

import argparse
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
import pandas as pd

torch.set_default_tensor_type('torch.DoubleTensor')

c:\Users\zhossai3\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [3]:
import pandas as pd

# Read CSV file into a Pandas DataFrame
groundTruth = pd.read_csv(r'C:\Users\zhossai3\Desktop\My code\OT_Fairness\Data\German Credit\German_Credit_Ground_Truth.csv', delimiter=',', header=0)

# Store feature columns in a DataFrame

X= groundTruth.iloc[:, :-1]  # Selects all rows and all columns except the last one
Y = groundTruth.iloc[:, -1]  # Selects all rows and only the last column

# Display the DataFrames
print(X)  # Print first few rows of features
 # Print first few rows of target variable



     checking-account  duration  credit-history  purpose  credit-amount  \
0                   1         6               0        7           1169   
1                   0        48               1        7           5951   
2                   3        12               0        4           2096   
3                   1        42               1        5           7882   
4                   1        24               2        1           4870   
..                ...       ...             ...      ...            ...   
995                 3        12               1        5           1736   
996                 1        30               1        2           3857   
997                 3        12               1        7            804   
998                 1        45               1        7           1845   
999                 0        45               0        2           4576   

     savings-account  employment-since  installment-rate  other-debtors  \
0                  4    

In [4]:
generator = Inject_Missing_Values()
miss_mcar30,index_mcar30 = generator.MCAR(X,missing_rate=30)
type(miss_mcar30)
miss_mcar30 = pd.concat([miss_mcar30, Y], axis=1)

In [5]:
miss_mcar30_numpy = miss_mcar30.to_numpy()
X_numpy = X.to_numpy()
Y_numpy = Y.to_numpy()

miss_mcar30_tensor = torch.tensor(miss_mcar30_numpy)
groundTruth_tensor = torch.tensor(groundTruth.to_numpy())
X_tensor = torch.tensor(X_numpy)
Y_tensor = torch.tensor(Y_numpy)

In [6]:
bucket_specs = {
     
    11: 4,   # Column 11 → age (4 bins)
    19: 2,  # Column 19 → sex (2 bins)
    21: 2,  # Column 21 → label  (2 bins)
    5: 5,   # Column 5 → savings-account (5 bins)
    15: 4   # Column 15 → jobs (4 bins)
}

# Define multiple attributes for X, Y, Z
X_cols = [11,19]  # Bucketized sensitive attributes (e.g., sex, race, age)
Y_cols = [21]     # Bucketized outcome-related attributes
Z_cols = [5, 15]  

In [ ]:
#sinkhorn
n_mcar30, d_mcar30 = miss_mcar30_tensor.shape
batchsize = 128 # If the batch size is larger than half the dataset's size,
                # it will be redefined in the imputation methods.
lr = 1e-2
epsilon_mcar30 = pick_epsilon(miss_mcar30_tensor)
#epsilon_mcar30 = 0.3
print(epsilon_mcar30)
mask_mcar30= torch.isnan(miss_mcar30_tensor).double()


sk_imputer_mcar30 = SinkhornImputation_CMI(eps=epsilon_mcar30, batchsize=batchsize, lr=lr, niter=2000)
sk_imp_mcar30, sk_maes30mcar, sk_rmses30mcar,cmi_loss = sk_imputer_mcar30.fit_transform(miss_mcar30_tensor, True, 50, groundTruth_tensor, X_cols, Y_cols, Z_cols, bucket_specs)
#using numpy version of data
sk_imp_mcar30_numpy = sk_imp_mcar30.detach().cpu().numpy()

sk_mae_mcar30 = MAE(sk_imp_mcar30,groundTruth_tensor , mask_mcar30)
sk_rmse_mcar30 = RMSE(sk_imp_mcar30, groundTruth_tensor, mask_mcar30)
print(sk_mae_mcar30,sk_rmse_mcar30)

60803.75256962414


INFO:root:Iteration 0:	 Loss: 52654.6970	 Validation MAE: 90.0804	RMSE: 560.0054


tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([0.0000, 0.7500, 1.5000, 2.2500, 3.0000])
tensor([1, 2, 3, 4])
tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([0.0000, 0.7500, 1.5000, 2.2500, 3.0000])
tensor([1, 2, 3, 4])
tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([0.0000, 0.7500, 1.5000, 2.2500, 3.0000])
tensor([1, 2, 3, 4])
tensor([20.0000, 33.7500, 47.5000,

INFO:root:Iteration 50:	 Loss: 3230.5967	 Validation MAE: 90.1026	RMSE: 560.0053


tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([0.0000, 0.6338, 1.2676])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([0.0000, 0.7500, 1.5000, 2.2500, 3.0000])
tensor([1, 2, 3, 4])
tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([0.0000, 0.6338, 1.2676])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([0.0000, 0.7500, 1.5000, 2.2500, 3.0000])
tensor([1, 2, 3, 4])
tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([0.0000, 0.6069, 1.2137])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([0.0000, 0.7500, 1.5000, 2.2500, 3.0000])
tensor([1, 2, 3, 4])
tensor([20.0000, 33.7500, 47.5000,

...

INFO:root:Iteration 1900:	 Loss: 14057.4460	 Validation MAE: 90.1133	RMSE: 560.0057


tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([-0.0308,  0.6546,  1.3400])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([-0.2147,  0.5889,  1.3926,  2.1963,  3.0000])
tensor([1, 2, 3, 4])
tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([-0.0308,  0.6546,  1.3400])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([-0.2525,  0.5607,  1.3738,  2.1869,  3.0000])
tensor([1, 2, 3, 4])
tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([-0.0308,  0.6546,  1.3400])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([-0.2525,  0.5607,  1.3738,  2.1869,  3.0000])
tensor([1, 2, 3, 4])
tensor([20

INFO:root:Iteration 1950:	 Loss: 27725.0609	 Validation MAE: 90.1139	RMSE: 560.0055


tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([0.0000, 0.6890, 1.3781])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([-0.3450,  0.4913,  1.3275,  2.1638,  3.0000])
tensor([1, 2, 3, 4])
tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([0.0000, 0.6890, 1.3781])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([-0.3450,  0.4913,  1.3275,  2.1638,  3.0000])
tensor([1, 2, 3, 4])
tensor([20.0000, 33.7500, 47.5000, 61.2500, 75.0000])
tensor([1, 2, 3, 4])
tensor([0.0000, 0.6890, 1.3781])
tensor([1, 2])
tensor([0.0000, 0.5000, 1.0000])
tensor([1, 2])
tensor([0.0000, 0.8000, 1.6000, 2.4000, 3.2000, 4.0000])
tensor([1, 2, 3, 4, 5])
tensor([-0.3590,  0.4808,  1.3205,  2.1603,  3.0000])
tensor([1, 2, 3, 4])
tensor([20.0000, 33

TypeError: abs(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
npoints = len(sk_maes30mcar)

ax.plot(np.linspace(0, 1, npoints) * npoints, sk_maes30mcar, label='Sinkhorn')
ax.set_xticks(np.linspace(0, npoints, 6))
ax.set_xlabel('# Sinkhorn iterations')

#ax.hlines(y=mean_mae_mcar60, xmin=0, xmax=npoints, label="Means", ls = '--', color='black')
#ax.hlines(y=ice_mae_mcar60, xmin=0, xmax=npoints, label="ICE", ls = '--', color='m')
#ax.hlines(y=soft_mae_mcar60, xmin=0, xmax=npoints, label="Softimpute", ls = '--', color='c')

ax.set_ylabel('MAE')
plt.legend()
plt.show()